In [1]:
from sgfmill import sgf
from enum import Enum
import numpy as np
import  matplotlib.pyplot as plt

In [2]:
train_data = open('Training Dataset/play_style_train.csv').read().splitlines()


In [3]:
# games = [i.split(',',1)[-1] for i in train_data]
# games[0]

In [4]:
train_data[0]

'PSL0000000001,1,B[dq],W[pd],B[qp],W[dd],B[cf],W[do],B[ck]'

In [5]:
#找棋盤每一個空地附近黑白子遠近

def space_far(board):

    space_farb = np.zeros((19,19), dtype=int)
    space_farw = np.zeros((19,19), dtype=int)

    for i in range(19):
        for j in range(19):
            if board[i][j] ==0:
                count = 1
                Find = False
                
                #找空地旁邊的子
                while (Find == False) and (count <= 19):
                    
                    arround = []
                    for dx in range(-1*count,count+1):
                        for dy in range(-1*count,count+1):
                            ni, nj = i + dx, j + dy
                            if 0 <= ni < 19 and 0 <= nj < 19:
                                
                                arround.append(board[ni][nj])
                                
                                
                                # if board[ni][nj] == 1:
                                #     space_farb[i][j] = count
                                # elif  board[ni][nj] == 2:
                                #     space_farb[i][j] = count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
                    # print(arround)
                    
                    if (space_farb[i][j] == 0) and (1 in arround):
                        space_farb[i][j] = count
                    
                    if (space_farw[i][j] == 0 ) and (2  in arround):
                        space_farw[i][j] = count
                        
                            
                    if (space_farb[i][j] != 0 )and (space_farw[i][j] != 0):
                        Find = True
                        
                    else:
                        count += 1     
                    
    return space_farb , space_farw

In [6]:
# 計算黑白棋氣數
def calculate_liberties(board):
    rows, cols = len(board), len(board[0])
    black_liberty = [[0 for _ in range(cols)] for _ in range(rows)]
    white_liberty = [[0 for _ in range(cols)] for _ in range(rows)]
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    def dfs(i, j, color):
        if i < 0 or i >= rows or j < 0 or j >= cols or visited[i][j] or board[i][j] != color:
            return []
        visited[i][j] = True
        group = [(i, j)]
        for dx, dy in directions:
            group.extend(dfs(i + dx, j + dy, color))
        return group
    
    def count_liberties(group):
        liberties = set()
        for i, j in group:
            for dx, dy in directions:
                ni, nj = i + dx, j + dy
                if 0 <= ni < rows and 0 <= nj < cols and board[ni][nj] == 0:
                    liberties.add((ni, nj))
        return len(liberties)
    
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j] and board[i][j] != 0:
                group = dfs(i, j, board[i][j])
                liberties = count_liberties(group)
                for x, y in group:
                    if board[i][j] == 1:
                        black_liberty[x][y] = liberties
                    else:
                        white_liberty[x][y] = liberties
    
    return black_liberty, white_liberty


# black_liberty_map, white_liberty_map = calculate_liberties(board)


In [7]:
def areamap(original_matrix_v5,range_11): # original_matrix_v5: 原始board, range_11:未來board
    for i in range(19):
        for j in range(19):
            # 檢查是不是0
            if original_matrix_v5[i][j] == 0:
                
                count2 = 0
                yes2 = True
                for k in range(1,6):
                    if i+k < 19:
                        if original_matrix_v5[i+k][j] == 2:
                            yes2 = False
                            break
                        elif original_matrix_v5[i+k][j] == 1:
                            count2 += 1
                            break
                for k in range(1,6):
                    if i-k >= 0:
                        if original_matrix_v5[i-k][j] == 2:
                            yes2 = False
                            break
                        elif original_matrix_v5[i-k][j] == 1:
                            count2 += 1
                            break        
                for k in range(1,6): 
                    if j+k < 19:
                        if original_matrix_v5[i][j+k] == 2:
                            yes2 = False
                            break
                        elif original_matrix_v5[i][j+k] == 1:
                            count2 += 1
                            break               
                for k in range(1,6):
                    if j-k >=0 :
                        if original_matrix_v5[i][j-k] == 2:
                            yes2 = False
                            break
                        elif original_matrix_v5[i][j-k] == 1:
                            count2 += 1
                            break
                if ( yes2== True) and (count2 > 0):
                    range_11[i][j] = 1 
                
                
                count1 = 0
                yes1 = True
                for k in range(1,6):
                    if i+k < 19:
                        if original_matrix_v5[i+k][j] == 1:
                            yes1 = False
                            break
                        elif original_matrix_v5[i+k][j] == 2:
                            count1 += 1
                            break    
                for k in range(1,6):    
                    if i-k >= 0:
                        if original_matrix_v5[i-k][j] == 1:
                            yes1 = False
                            break
                        elif original_matrix_v5[i-k][j] == 2:
                            count1 += 1
                            break        
                for k in range(1,6):  
                    if j+k < 19:
                        if original_matrix_v5[i][j+k] == 1:
                            yes1 = False
                            break
                        elif original_matrix_v5[i][j+k] == 2:
                            count1 += 1
                            break                      
                for k in range(1,6):
                    if j-k >=0 :
                        if original_matrix_v5[i][j-k] == 1:
                            yes1 = False
                            break
                        elif original_matrix_v5[i][j-k] == 2:
                            count1 += 1
                            break                               
                if ( yes1== True) and (count1 > 0):
                    range_11[i][j] = 2 
                
        
    return  range_11                        

                
                

In [8]:
#計算有沒有死掉的棋
def check_and_remove_dead_stones(board, x, y):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    rows, cols = board.shape

    def dfs(i, j, color):
        if i < 0 or i >= rows or j < 0 or j >= cols or visited[i][j] or board[i][j] != color:
            return []
        visited[i][j] = True
        group = [(i, j)]
        for dx, dy in directions:
            group.extend(dfs(i + dx, j + dy, color))
        return group

    def count_liberties(group):
        liberties = set()
        for i, j in group:
            for dx, dy in directions:
                ni, nj = i + dx, j + dy
                if 0 <= ni < rows and 0 <= nj < cols and board[ni][nj] == 0:
                    liberties.add((ni, nj))
        return len(liberties)

    visited = np.zeros((rows, cols), dtype=bool)
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and board[nx][ny] != 0 and not visited[nx][ny]:
            group = dfs(nx, ny, board[nx][ny])
            if count_liberties(group) == 0:
                for i, j in group:
                    board[i][j] = 0  # Remove dead stones

In [9]:
def sgf_to_array(sgf_content):
    
    sgf_X = []
    # sgf_Y = []
    sgf_game = sgf.Sgf_game.from_bytes(sgf_content)
    board_size = 19

    random_integers  = [len(sgf_game.get_main_sequence())-1]
    for idx,num in enumerate(random_integers):
        
        X = []
        # Y = []    
        board = np.zeros((19,19), dtype=int)
        pre_step = np.zeros((19,19), dtype=int)

        #最後一步num
        for node in sgf_game.get_main_sequence()[-1:]: 
            move = node.get_move()
            # print(move)
            if move is not None:
                color, (x, y) = move
                
                # 將這一步棋作為目標（Y）添加
                last_move = np.zeros((19, 19), dtype=int)
                last_move[x, y] = 1            
                X.append(last_move.copy())
                

        # 黑白棋勢力
        for node in sgf_game.get_main_sequence()[1:num+1]:  # 跳過根節點
            move = node.get_move()            
            if move is not None:
                color, (x, y) = move
                board_value = 1 if color == 'b' else 2  # 1 for black, 2 for white
                board[x, y] = board_value
                #死子確認和移除
                check_and_remove_dead_stones(board, x, y)     
                    
        area = board.copy()
        # print(area)
        #空格
        X.append((board == 0).astype(int))
        # 黑白棋位置
        X.append((board == 1).astype(int))
        X.append((board == 2).astype(int))
        
                
        # 計算黑白棋氣數
        BLM,WLM = calculate_liberties(board.copy())
        # print(BLM)
        BLM_ =  [[-num for num in sublist] for sublist in BLM]
        # print(BLM_)
        # print(BLM_ / np.linalg.norm(BLM_))
        WLM_ =  [[-num for num in sublist] for sublist in WLM]
        X.append(BLM_ )#/ np.linalg.norm(BLM_))
        X.append(WLM_ )#/ np.linalg.norm(WLM_))
        
        
        #黑白棋腹地(簡易版)
        Area = areamap(board,board.copy())
        # Area = areamap(Area,Area.copy())
        X.append((Area == 1).astype(int))
        X.append((Area == 2).astype(int))
        
        
        #找棋盤每一個空地附近黑白子遠近
        SFB ,SFW = space_far(board.copy())
        X.append(SFB) 
        X.append(SFW) 
        


        sgf_X.append(X)
        # sgf_Y.append(Y)
        
    return np.array(sgf_X) #, np.array(sgf_Y) ,np.array(random_integers)
        # return board
        

In [ ]:
'''這這這'''

X_train = []
y_train = []

for idx , i in enumerate(train_data):
    step_list = train_data[idx].split(',')
    train_sgf_name = 'organize/Play_Style/_rename_SGF/PS_train_' +  step_list[0] + '_'  + f"{idx:05}" + '_' + 'Style' + step_list[1] + '.sgf'
    f = open(train_sgf_name, 'w')
    f.write('(;GM[1]FF[4]SZ[19]')
    
    for step in step_list[2:]:
        f.write(';')
        f.write(step)
    
                
    f.write(')')
    f.close()    
    
    # train_sgf_name
    
    with open(train_sgf_name, "rb") as f:
        sgf_content = f.read()
    X = sgf_to_array(sgf_content)
    X = np.array(X)
    X_transposed_array = X.transpose(( 1, 2 ,0))
    
    X_train.append(X_transposed_array)
    y_train.append(int(step_list[1]))
    

In [16]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train.shape,y_train.shape

((26615, 19, 19, 10), (26615,))

In [17]:
train_npz_path = 'E:/research/AIcup_GO/TrainingData/organize/Play_Style/NPZ/PS_train_dataset3_feature10_v3.npz'
np.savez(train_npz_path, x_train=X_train, y_train=y_train)